This notebook is used to fetch 'token transactions' and 'normal transactions' from Etherscan for all wallets that received payment from Storj at least once in the 6 months prior to May 2021 (excluding the approx. 389 zksync wallets).-- Lisa Marshall

In [1]:
import pandas as pd
import requests
import time
import datetime as dt
import csv
import json

In [2]:
#Create dataframes to hold the data -- one for token transactions and one for normal transactions
dfToken = pd.DataFrame(columns=['blockNumber','timeStamp','hash','nonce','blockHash','from','contractAddress','to','value',
                           'tokenName','tokenSymbol','tokenDecimal','transactionIndex','gas','gasPrice','gasUsed',
                           'cumulativeGasUsed','input','confirmations'
])

dfNormal = pd.DataFrame(columns=['blockNumber','timeStamp','hash','nonce','blockHash','transactionIndex','from',
                                'to','value','gas','gasPrice','isError','txreceipt_status','input','contractAddress',
                                'cumulativeGasUsed','confirmations'])

In [13]:
#Since we've previously collected token transactions (June 19, 2021), use these instead.
#Note that this dataset has been processed via the script 'LimitTokenTXtoUsedWallets.ipynb', which
#limits the set to just the wallets that have been paid in the 6 months prior to May 2021,
#(minus the zksync wallets), and includes an extra column identifying wallets that are associated
#with other wallets -- ie, they've performed token transactions with each other.
dfToken2 = pd.read_csv('tokenTXS.csv')

#### Data last fetched on July 9, 2021

In [3]:
#Load the node operator data
operatorData = pd.read_pickle('operatorDatax.pkl')
len(operatorData)

5710

In [4]:
#remove the 21 rows that have NaNs for the unique emails and unique nodes 
#(these are some of the zksync wallets we couldn't identify)
operatorData = operatorData[~operatorData.unique_emails.isna()]
len(operatorData)

5689

In [5]:
#Create a list of paid wallets
paidWallets = operatorData['wallet'].unique()
len(paidWallets)

5689

In [6]:
#Create list of wallets with zksync rather than ethereum addresses (these need to be removed)
zksyncWallets = pd.read_csv('missingWallets.csv')
zksync = zksyncWallets.wallet.unique()
len(zksync)

397

In [8]:
#Remove the zksync wallets
import numpy as np
toFetch = np.setdiff1d(paidWallets,zksync)

In [9]:
t=0
error_wallets=[]

In [30]:
#for each node operator in the wallet list, fetch their token transfer txn history (sleeping for 5 seconds between each fetch)
#fetch sections of toFetch at a time to avoid losing work if computer crashes
for wallet in toFetch[4499:]:
    #url = 'https://api.etherscan.io/api?module=account&action=tokentx&address={}&startblock=0&endblock=99999999&sort=asc&apikey=VZPN8DRQK4FXXURQKGW6P1XG4Z8SMKZE1K'.format(wallet)
    url = 'https://api.etherscan.io/api?module=account&action=txlist&address={}&startblock=0&endblock=99999999&sort=asc&apikey=VZPN8DRQK4FXXURQKGW6P1XG4Z8SMKZE1K'.format(wallet)
    mssg = 'OK'
    response = requests.get(url)
    jsonOP = response.json()
    parse2 = jsonOP['status']
    if parse2 == '0':
        mssg = jsonOP['message']
        if mssg == "No transactions found":
            pass
        else:
            error_wallets = error_wallets.append(wallet)
    else:
        parse1 = jsonOP['result']
        for i in parse1:
            #dfToken = dfToken.append(i, ignore_index=True)
            dfNormal = dfNormal.append(i, ignore_index=True)
    time.sleep(1.1)
    print(t, 'status: ',parse2,'message: ',mssg)
    t+=1
        

4499 status:  1 message:  OK
4500 status:  1 message:  OK
4501 status:  0 message:  No transactions found
4502 status:  1 message:  OK
4503 status:  0 message:  No transactions found
4504 status:  1 message:  OK
4505 status:  0 message:  No transactions found
4506 status:  1 message:  OK
4507 status:  0 message:  No transactions found
4508 status:  0 message:  No transactions found
4509 status:  0 message:  No transactions found
4510 status:  1 message:  OK
4511 status:  1 message:  OK
4512 status:  1 message:  OK
4513 status:  1 message:  OK
4514 status:  1 message:  OK
4515 status:  1 message:  OK
4516 status:  1 message:  OK
4517 status:  1 message:  OK
4518 status:  1 message:  OK
4519 status:  0 message:  No transactions found
4520 status:  1 message:  OK
4521 status:  0 message:  No transactions found
4522 status:  1 message:  OK
4523 status:  0 message:  No transactions found
4524 status:  1 message:  OK
4525 status:  1 message:  OK
4526 status:  0 message:  No transactions foun

4730 status:  1 message:  OK
4731 status:  1 message:  OK
4732 status:  0 message:  No transactions found
4733 status:  0 message:  No transactions found
4734 status:  1 message:  OK
4735 status:  1 message:  OK
4736 status:  1 message:  OK
4737 status:  1 message:  OK
4738 status:  1 message:  OK
4739 status:  1 message:  OK
4740 status:  1 message:  OK
4741 status:  1 message:  OK
4742 status:  1 message:  OK
4743 status:  0 message:  No transactions found
4744 status:  0 message:  No transactions found
4745 status:  1 message:  OK
4746 status:  0 message:  No transactions found
4747 status:  1 message:  OK
4748 status:  0 message:  No transactions found
4749 status:  1 message:  OK
4750 status:  1 message:  OK
4751 status:  1 message:  OK
4752 status:  0 message:  No transactions found
4753 status:  1 message:  OK
4754 status:  1 message:  OK
4755 status:  0 message:  No transactions found
4756 status:  1 message:  OK
4757 status:  1 message:  OK
4758 status:  1 message:  OK
4759 st

4961 status:  0 message:  No transactions found
4962 status:  0 message:  No transactions found
4963 status:  1 message:  OK
4964 status:  1 message:  OK
4965 status:  1 message:  OK
4966 status:  1 message:  OK
4967 status:  0 message:  No transactions found
4968 status:  1 message:  OK
4969 status:  1 message:  OK
4970 status:  1 message:  OK
4971 status:  0 message:  No transactions found
4972 status:  0 message:  No transactions found
4973 status:  0 message:  No transactions found
4974 status:  1 message:  OK
4975 status:  1 message:  OK
4976 status:  1 message:  OK
4977 status:  0 message:  No transactions found
4978 status:  0 message:  No transactions found
4979 status:  1 message:  OK
4980 status:  0 message:  No transactions found
4981 status:  0 message:  No transactions found
4982 status:  1 message:  OK
4983 status:  1 message:  OK
4984 status:  0 message:  No transactions found
4985 status:  0 message:  No transactions found
4986 status:  1 message:  OK
4987 status:  1 me

5192 status:  1 message:  OK
5193 status:  0 message:  No transactions found
5194 status:  1 message:  OK
5195 status:  0 message:  No transactions found
5196 status:  0 message:  No transactions found
5197 status:  1 message:  OK
5198 status:  0 message:  No transactions found
5199 status:  0 message:  No transactions found
5200 status:  1 message:  OK
5201 status:  0 message:  No transactions found
5202 status:  1 message:  OK
5203 status:  1 message:  OK
5204 status:  1 message:  OK
5205 status:  1 message:  OK
5206 status:  0 message:  No transactions found
5207 status:  1 message:  OK
5208 status:  1 message:  OK
5209 status:  1 message:  OK
5210 status:  1 message:  OK
5211 status:  1 message:  OK
5212 status:  1 message:  OK
5213 status:  1 message:  OK
5214 status:  1 message:  OK
5215 status:  1 message:  OK
5216 status:  0 message:  No transactions found
5217 status:  1 message:  OK
5218 status:  0 message:  No transactions found
5219 status:  0 message:  No transactions foun

In [32]:
len(dfNormal)

201276

In [33]:
dfNormal.to_csv('dfNormal.csv')